## Plots pandas

In [ ]:
%run ../../lib/basic-import.ipynb
%run ../../lib/plot-functions.ipynb

In [ ]:
#Brukes til -30 + 30 sek for stops.
indexArray = []
tempIndexes = []
tempStops=[]

#kanskje endre til -29 +29.
def lookup_dfs(df, i, z):
    stopStart = df[i].index.get_loc(indexArray[i][z])
    tempDF = df[i].iloc[stopStart+1:]
    stopEnd = df[i].index.get_loc(tempDF[(tempDF['stop'].diff() != 0) & (tempDF['lap'].diff() == 0) & (tempDF['lap'] == z+1)].index[0])
    return df[i].iloc[stopStart-30:stopEnd+30]
    

#bruker -1 , men MÅ endre. Gjør pga stuck, siste måling virker spesiell..
for i in range(len(all_stops)):
    for x in range(len(all_stops[i])-1):
        tempIndexes.append(all_stops[i].index[x])
    indexArray.append(tempIndexes)
    tempIndexes = []
    for z in range(len(indexArray[i])):
        tempStops.append(lookup_dfs(all_dfs, i, z))
    userStop = pd.concat(tempStops)
    all_dStops.append(userStop)
    tempStops = []

In [ ]:
#f = open('all_dStops.txt', 'w')
#for element in all_dStops:
    #f.write(element.to_string())
    #f.write('\n')
#f.close()

In [ ]:
#Axis for 10 minutes start for every participant.
add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])

for participant_file in participant_files:
    rightPosition = 1.00
    leftPosition = -0.10
    #first ax and figure stretch.
    ID = participant_file[:-4]
    fig, ax = plt.subplots()
    fig.subplots_adjust(right=2)
    #number of additional y axis:
    ax2 = ax.twinx()
    ax3 = ax.twinx()
    ax4 = ax.twinx()
    ax5 = ax.twinx()

    #position spines and ticks for axis.
    rightPosition = ax_modification(ax2,'right', rightPosition)
    rightPosition = ax_modification(ax3,'right', rightPosition)
    leftPosition = ax_modification(ax4,'left', leftPosition)
    rightPosition = ax_modification(ax5,'right', rightPosition)


    #Finds start where not only NaN:
    startDate = add_df[ID].index[~add_df[ID].isnull().all(1)]
    startRow = add_df[ID].index.get_loc(startDate[0])    
    endRow = startRow + 600

    add_df[ID]['accumulated_power'].iloc[startRow:endRow].plot(ax=ax, color='red',legend=False, title= ID + " 10 minutes start")
    add_df[ID]['speed'].iloc[startRow:endRow].plot(ax=ax2, color='blue',legend=False)
    add_df[ID]['power'].iloc[startRow:endRow].rolling(window=30).mean().plot(ax=ax3, color='green',legend=False)
    add_df[ID]['altitude'].iloc[startRow:endRow].plot(ax=ax4, color='orange',legend=False)
    add_df[ID]['distance'].diff().iloc[startRow:endRow].plot(ax=ax5, color='lightcoral',legend=False)

    ax_style(ax,'Accumulated Power','red')
    ax_style(ax2,'Speed','blue')
    ax_style(ax3,'MA Power','green')
    ax_style(ax4,'Altitude','orange')
    ax_style(ax5,'Distance change','lightcoral')

    ax5.legend([ax.get_lines()[0], ax2.get_lines()[0], ax3.get_lines()[0], ax4.get_lines()[0], ax5.get_lines()[0]], ['Accumulated Power','Speed','MA Power','Altitude', 'Distance change'], loc='lower left', bbox_to_anchor=(1.5, 0.5))

In [ ]:
add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])

for participant_file in participant_files:
    rightPosition = 1.00
    leftPosition = -0.10

    ID = participant_file[:-4]
    fig, ax = plt.subplots()
    fig.subplots_adjust(right=2)

    ax2 = ax.twinx()
    ax3 = ax.twinx()
    ax4 = ax.twinx()

    rightPosition = ax_modification(ax2,'right', rightPosition)
    rightPosition = ax_modification(ax3,'right', rightPosition)
    leftPosition = ax_modification(ax4,'left', leftPosition)

    add_df_plot = add_df[ID][~add_df[ID].isnull().all(1)]

    add_df_plot['heart_rate'].plot(ax=ax, color='red',legend=False, title= ID + " Relativ Start")
    add_df_plot['speed'].plot(ax=ax2, color='blue',legend=False)
    add_df_plot['power'].rolling(window=30).mean().plot(ax=ax3, color='green',legend=False)
    add_df_plot['distance'].plot(ax=ax4, color='orange',legend=False)

    ax_style(ax,'Heart rate','red')
    ax_style(ax2,'Speed','blue')
    ax_style(ax3,'MA Power','green')
    ax_style(ax4,'Distance','orange')

    ax4.legend([ax.get_lines()[0], ax2.get_lines()[0], ax3.get_lines()[0], ax4.get_lines()[0]], ['Heart rate','Speed','MA Power','Distance'], loc='lower left', bbox_to_anchor=(1.3, 0.5))


In [ ]:
all_dStops_df = pd.concat(all_dStops, axis=1, keys=[a[:-4] for a in participant_files])
all_dStops_df.index = map(lambda x: str(x)[-8:], all_dStops_df.index)

for participant_file in participant_files:
    rightPosition = 1.00
    leftPosition = -0.10
    ID = participant_file[:-4]

    fig, ax = plt.subplots()
    fig.subplots_adjust(right=2)

    ax2 = ax.twinx()
    ax3 = ax.twinx()
    ax4 = ax.twinx()
    ax5 = ax.twinx()
    ax6 = ax.twinx()

    rightPosition = ax_modification(ax2,'right', rightPosition)
    rightPosition = ax_modification(ax3,'right', rightPosition)
    rightPosition = ax_modification(ax4, 'right', rightPosition)
    leftPosition = ax_modification(ax5,'left', leftPosition)
    leftPosition = ax_modification(ax6,'left', leftPosition)

    all_plot_dstops = all_dStops_df[ID][~all_dStops_df[ID].isnull().all(1)]
    all_plot_dstops.iloc[-1:, [all_plot_dstops.columns.get_loc(col) for col in fitgen_cols[1:]]]=np.NaN

    
    all_plot_dstops['heart_rate'].plot(ax=ax, color='red',legend=False, title= ID + " All stops")
    all_plot_dstops['speed'].plot(ax=ax2, color='blue',legend=False)
    all_plot_dstops['power'].rolling(window=30).mean().plot(ax=ax3, color='green',legend=False)
    all_plot_dstops['distance'].diff().plot(ax=ax4, color='orange',legend=False)
    all_plot_dstops['altitude'].plot(ax=ax5, color='lightcoral',legend=False)
    all_plot_dstops['accumulated_power'].plot(ax=ax6, color='purple',legend=False)
    
    ax_style(ax,'Heart rate','red')
    ax_style(ax2,'Speed','blue')
    ax_style(ax3,'MA Power','green')
    ax_style(ax4,'Distance change','orange')
    ax_style(ax5,'Altitude','lightcoral')
    ax_style(ax6,'Accumulated Power','purple')

    ax6.legend([ax.get_lines()[0], ax2.get_lines()[0], ax3.get_lines()[0], ax4.get_lines()[0],ax5.get_lines()[0], ax6.get_lines()[0]], ['Heart rate','Speed','MA Power','Distance change','Altitude','Accumulated Power'], loc='lower left', bbox_to_anchor=(1.5, 0.5))

In [ ]:
all_dStops_df = pd.concat(all_dStops, axis=1, keys=[a[:-4] for a in participant_files])

for participant_file in participant_files:
    rightPosition = 1.00
    leftPosition = -0.10
    ID = participant_file[:-4]

    fig, ax = plt.subplots()
    fig.subplots_adjust(right=2)

    ax2 = ax.twinx()
    ax3 = ax.twinx()
    ax4 = ax.twinx()
    ax5 = ax.twinx()
    ax6 = ax.twinx()

    rightPosition = ax_modification(ax2,'right', rightPosition)
    rightPosition = ax_modification(ax3,'right', rightPosition)
    rightPosition = ax_modification(ax4, 'right', rightPosition)
    leftPosition = ax_modification(ax5,'left', leftPosition)
    leftPosition = ax_modification(ax6,'left', leftPosition)

    #finding start and endDate for vertical line. Rows for iloc.
    descendingDF = all_dStops_df[ID].sort_index(ascending=False)
    startDate = all_dStops_df[ID]['lap'].idxmax()
    endDate = descendingDF['lap'].idxmax() - pd.Timedelta(seconds=30)

    startRow = all_dStops_df[ID].index.get_loc(startDate)-31
    endRow = all_dStops_df[ID].index.get_loc(endDate)+30


    all_dStops_df[ID][['heart_rate']].iloc[startRow:endRow].plot(ax=ax, color='red',legend=False, title= ID + " Last stop")
    all_dStops_df[ID][['speed']].iloc[startRow:endRow].plot(ax=ax2, color='blue',legend=False)
    all_dStops_df[ID][['power']].iloc[startRow:endRow].plot(ax=ax3, color='green',legend=False)
    all_dStops_df[ID][['distance']].iloc[startRow:endRow].diff().plot(ax=ax4, color='orange',legend=False)
    all_dStops_df[ID][['altitude']].iloc[startRow:endRow].plot(ax=ax5, color='lightcoral',legend=False)
    all_dStops_df[ID][['accumulated_power']].iloc[startRow:endRow].plot(ax=ax6, color='purple',legend=False)

    ax_style(ax,'Heart rate','red')
    ax_style(ax2,'Speed','blue')
    ax_style(ax3,'MA Power','green')
    ax_style(ax4,'Distance change','orange')
    ax_style(ax5,'Altitude','lightcoral')
    ax_style(ax6,'Accumulated Power','purple')

    ax.axvline(startDate, color='black', linestyle='--', lw=2)
    ax.axvline(endDate, color='black', linestyle='--', lw=2)

    ax6.legend([ax.get_lines()[0], ax2.get_lines()[0], ax3.get_lines()[0], ax4.get_lines()[0],ax5.get_lines()[0], ax6.get_lines()[0]], ['Heart rate','Speed','Power','Distance change','Altitude','Accumulated Power'], loc='lower left', bbox_to_anchor=(1.5, 0.5))